### This code extracts a garnet composition from the garnet data file, reformats the wt% composition, calculates the mol oxide composition, and in turn get's the endmember composition

In [ ]:
import thermoengine as thermo

import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
from collections import OrderedDict
%matplotlib notebook

In [ ]:
data_dir="/Users/jennaadams/Documents/projects/garnet-soln/data/"

lepr_data=pd.read_excel(data_dir+'grt_bearing_expts.xls',sheetname=None)

In [ ]:
experiments = lepr_data.pop('Experiment')
exp_data = experiments.filter(['Index', 'T (C)', 'err T (C)', 'P (GPa)','err P (GPa)', 'fO2 cond', 
                                                 'Wt: SiO2', 'Wt: TiO2','Wt: Al2O3', 'Wt: Fe2O3', 'Wt: Cr2O3', 
                                                 'Wt: FeO', 'Wt: MnO', 'Wt: MgO','Wt: NiO', 'Wt: CaO', 'Wt: Na2O', 
                                                 'Wt: K2O', 'Wt: P2O5', 'Wt: H2O', 'Wt: CO2'], axis=1)

metadata = experiments.filter(['Index','Experiment', 'Author (year)', 'Laboratory', 'Device', 'Container', 'Method', 
                               'Duration_hours', 'Phases'], axis=1)



In [ ]:
exp_data.columns

In [ ]:
metadata.columns

In [ ]:
phs_symbols = lepr_data.pop('phase_symbols')
    
phs_data = lepr_data

In [ ]:
garnet_data = rxn_data['Garnet']

oxide_names = thermo.chem.oxide_props['oxides']

In [ ]:
oxide_names

In [ ]:
garnet_data.dropna(subset=['Index'], inplace=True)

In [ ]:
garnet_data['Index'] = garnet_data['Index'].astype(int)

In [ ]:
garnet_data.set_index('Index', inplace=True)

In [ ]:
garnet_data.head()

In [ ]:
garnet_comp = garnet_data.loc[[337],['Wt: SiO2', 'Wt: TiO2', 'Wt: Al2O3', 'Wt: Fe2O3', 
                                     'Wt: Cr2O3', 'Wt: FeO', 'Wt: MnO', 'Wt: MgO', 'Wt: NiO', 
                                     'Wt: CoO', 'Wt: CaO', 'Wt: Na2O', 'Wt: K2O', 'Wt: P2O5', 'Wt: H2O', 'Wt: CO2']]

In [ ]:
garnet_comp.fillna(0, inplace=True)

In [ ]:
garnet_comp

In [ ]:
garnet_wt_comp = np.array(garnet_comp)[0]

In [ ]:
garnet_wt_comp

In [ ]:
#Trying to take a row in excel file and filter columns to those that match only the oxide_order elements

#oxide_order = thermo.chem.OXIDE_ORDER

#garnet_comp2=[]
#for ioxide in oxide_order:
    #garnet_comp2 = garnet_comp.isin(['Wt:' + ioxide])
    
#garnet_comp2

In [ ]:
#garnet_wt_comp = garnet_comp.to_dict('records', into=OrderedDict)

In [ ]:
#garnet_wt_comp

In [ ]:

#garnet_wt_comp['SiO2'] = garnet_wt_comp.pop('Wt: SiO2')
#garnet_wt_comp

In [ ]:
garnet_mol_oxide = thermo.chem.wt_to_mol_oxide(garnet_wt_comp)
garnet_mol_oxide

In [ ]:
garnet_mol_comp_dict = OrderedDict()
for iname, ioxide in zip(oxide_names, garnet_mol_oxide):
    garnet_mol_comp_dict[iname] = ioxide
    
garnet_mol_comp_dict

In [ ]:
garnet_mol_comp_array=thermo.chem.format_mol_oxide_comp(garnet_mol_comp_dict)

In [ ]:
garnet_mol_comp_array

In [ ]:
modelDB = thermo.model.Database()
phs = modelDB.get_phase('Grt')

In [ ]:
phs

In [ ]:
phs.calc_endmember_comp(garnet_mol_comp_array)

### Code below tests calc_endmember_comp; it appears to not produce correct results for almandine composition

In [ ]:
oxide_names = ['SiO2','TiO2','Al2O3','Fe2O3','Cr2O3','FeO','MnO','MgO','NiO',
        'CoO','CaO','Na2O','K2O','P2O5','H2O','CO2']
oxide_wts = [42.04, 0.71, 22.21, 0, 0.5, 8.78, 0, 20.48, 0, 0, 4.76, 0.53, 0, 0, 0, 0]
oxide_molecular_wts = thermo.chem.oxide_props['molwt']

In [ ]:
oxide_wts2 = [42.04, 0, 22.21, 0, 0, 8.78, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
mol_oxides = oxide_wts2/oxide_molecular_wts
mol_oxides = mol_oxides/np.sum(mol_oxides)
#mol_oxides = thermo.chem.wt_to_mol_oxide(oxide_names, oxide_wts)

mol_comp2 = OrderedDict()
for iname, ioxide in zip(oxide_names, mol_oxides):
    mol_comp2[iname] = ioxide
    
   

In [ ]:
mol_comp2

In [ ]:
mol_oxides

In [ ]:
mol_comp_mix=thermo.chem.format_mol_oxide_comp(mol_comp2)

In [ ]:
mol_comp_mix

In [ ]:
thermo.chem.OXIDE_ORDER

In [ ]:
modelDB = thermo.model.Database()

In [ ]:
phs = modelDB.get_phase('Grt')

In [ ]:
phs.calc_endmember_comp(mol_comp_mix)

In [ ]:
modelDB = thermo.model.Database()
phs = modelDB.get_phase('Grt')

In [ ]:
phases_present = rxn_data['Experiment']['Phases'][1]

### This code is old code that likely will no longer be used; it originally filtered phase assemblages from the garnet excel file for only unique phase assemblages; we are no longer implementing this idea

In [ ]:
unique_phase_symbols = ['Cpx', 'Ol', 'Grt']
rxn_svd_props = thermo.chem.calc_reaction_svd(unique_phase_symbols, TOLsvd=1e-4, modelDB=modelDB)
rxn_svd = rxn_svd_props['rxn_svd']
Nbasis=len(rxn_svd)
rxn_endmember_name = rxn_svd_props['all_endmember_name']
rxn_phase_symbols = rxn_svd_props['all_phase_symbol']

In [ ]:
unique_phase_assemblages = rxn_data['Experiment']['Phases'].unique()

In [ ]:
phase_assemblages=[]
for iphase_assem in unique_phase_assemblages:
    
    try: 
        phase_list = iphase_assem.split(sep='+')
        phase_list.sort()
        phase_assemblages.append(phase_list)
        
    except: 
        print('empty_assemblage')
        
phase_assemblages = np.unique(phase_assemblages)

In [ ]:
mask = []
for iassemblage in phase_assemblages:
    imask = 'liq' not in iassemblage
    
    mask.append(imask)
    
mask = np.array(mask)
np.sum(mask)

In [ ]:
phase_assemblages[mask]

In [ ]:
wtcoefs, costs, rxn_coefs, wtcoefs_ortho = thermo.chem.get_rxns(rxn_svd, Ndraw=3, ortho_scale=15, Nbasis=Nbasis, TOL=1e-10)

In [ ]:
(np.place(rxn_coefs, abs(rxn_coefs)< 1e-2, 0))

In [ ]:
endmember_ids = rxn_svd_props['all_endmember_id']
rxns = []
for irxn_coefs in rxn_coefs:
    irxn = modelDB.get_rxn(rxn_phase_symbols, endmember_ids, irxn_coefs)
    
    rxns.append(irxn)  

In [ ]:
phases = []

for irxn in rxns:
    phases.extend(irxn.phases)
    
phases = np.unique(phases)